# How to Train YOLOv5 on Custom Objects

https://models.roboflow.com/object-detection/yolov5

https://colab.research.google.com/drive/1gDZ2xcTOgR39tGGs-EZ6i3RTs16wmzZQ

이 가이드에서는 다음 단계를 수행한다.

1. YOLO 종속성 설치
2. 맞춤형 YOLO 물체 감지 데이터 다운로드
3. YOLO에 대해 사전 훈련된 가중치 준비
4. YOLO 교육 실행
5. YOLO 성능 평가
6. YOLO 훈련 데이터 시각화
7. 테스트 이미지에서 YOLO 추론 실행
8. 향후 추론을 위해 저장된 YOLO 가중치 내보내기

# 1. Install yolov5

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

개발 환경을 설정하려면 먼저 기본 YOLOV5 저장소를 복제하고, 필요한 요구 사항을 다운로드한다.

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 2. Download Correctly Formatted Custom Dataset 



![YOLOv5 PyTorch export](https://i.imgur.com/5vr9G2u.png)


https://public.roboflow.com/

## yolov5 디렉토리에 hardhat dataset을 다운로드 한다.


- train과 test만 있으므로(valid가 없음), test이미지 데이터를 val 이미지 데이터로 사용한다.

- 현재 의 디렉토리 위치를 확인

In [ ]:
%pwd

- roboflow에서 hardhat을 다운로드 한다.

In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")
!curl -L "https://public.roboflow.com/ds/CPjd8hWKob?key=EKt26DZnBM" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# 3. data.yaml에 뭐가 들어있나?

In [ ]:
%cat data.yaml

## 경로가 잘못되어 있으므로, yolov5에 있는 data.yaml을 열고 올바르게 수정한다.

## train: /content/yolov5/train/images
## val: /content/yolov5/test/images

## nc: 3
## names: ['head', 'helmet', 'person']

# 4. models에 있는 yolov5s.yaml의 내용을 확인하고 수정한다.

- nc의 값을 80에서 3으로 수정하기 위한 코딩 작업

In [6]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

- yolov5s.yaml의 내용을 확인한다.

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

-template 명령어가 실행 가능하도록 한다.

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# 5. nc가 3개이므로 yolov5s.yaml의 내용을 수정하여 custom_yolov5s.yaml을 만들고,
# 여기에다 위 yolov5s.yaml 의 내용을 복사해 넣고, 
# 80을 {num_classes}로 수정한다.

- %%writetemplate 명령어를 수행하여, models에 custom_yolov5s.yaml을 생성한다.

In [9]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## custom_yolov5s.yaml의 내용이 정확하게 수정되었는지 확인한다.

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

## data.yaml의 내용이 정확하게 수정되었는지 확인한다.

In [ ]:
%cat /content/yolov5/data.yaml

# 6. Train 

훈련을 시작하기 위해 다음 옵션을 사용하여 훈련 명령을 실행.

1. img: 입력 이미지 크기 정의
2. batch: 배치 크기 결정
3. epochs: 훈련 epoch의 수를 정의합니다. (참고: 종종 3000+가 일반적입니다!)
4. data: yaml 파일의 경로를 설정합니다(Roboflow에서 데이터 세트를 다운로드할 때 제공됨).
5. cfg: 모델 구성 지정
6. weights: 가중치 에 대한 사용자 정의 경로를 지정합니다. (참고: 위에서 다운로드한 사전 훈련된 가중치를 셸 스크립트로 지정할 수 있습니다)
7. name: 결과 이름
8. hyp: 훈련을 위한 하이퍼파라미터 정의

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data /content/yolov5/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# 7. Evaluate Custom YOLOv5 Detector Performance

tensorboard를 통해 폴더 의 로그를 시각화하여 훈련 절차가 얼마나 잘 수행되었는지 평가할 수 있다 .

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

Colab에서 직접 결과를 간단히 플롯할 수도 있다.

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

### 레이블된 이미지 데이터를 출력


- 원본 이미지 출력

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/test_batch0_labels.jpg', width=900)

- 증강된 이미지 출력

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

# 8. Run Inference  With Trained Weights


- test 이미지 추론하기

In [ ]:
# trained weights are saved by default in our weights folder
%ls /content/yolov5/runs/train/yolov5s_results/weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/test/images

- detect된 이미지를 확인해보자.

In [ ]:
Image(filename='/content/yolov5/runs/detect/exp2/005298_jpg.rf.7d1070e4a462d4c636a68e64f2013761.jpg', width=900)